In [ ]:
from rich.console import Console

import canonical_toolkit as ctk

In [ ]:
console = Console()

In [ ]:

import numpy as np
from networkx import DiGraph
from ariel.body_phenotypes.robogen_lite.config import (
    NUM_OF_FACES,
    NUM_OF_ROTATIONS,
    NUM_OF_TYPES_OF_MODULES,
)
from ariel.body_phenotypes.robogen_lite.decoders.hi_prob_decoding import (
    HighProbabilityDecoder,
)

SEED = 42
RNG = np.random.default_rng(SEED)


def generate_random_individual(
    num_modules: int = 20,
    seed: int | None = None,
) -> DiGraph:
    """
    Generate a random modular individual as a directed graph.

    Parameters
    ----------
    num_modules : int, default 20
        Number of modules to include in the generated individual.
    seed : int, default SEED
        Random seed for reproducible generation.

    Returns
    -------
    DiGraph
        A directed graph representing the randomly generated modular
        individual with modules, connections, and rotations.

    Notes
    -----
    - Uses three probability spaces: module types, connections between
      faces, and rotations
    - Probability matrices have shapes (num_modules, NUM_OF_TYPES_OF_MODULES),
      (num_modules, num_modules, NUM_OF_FACES), and
      (num_modules, NUM_OF_ROTATIONS) respectively
    - HighProbabilityDecoder converts probability matrices to graph structure
    """
    rng = np.random.default_rng(seed) if seed else RNG
    type_probability_space = rng.random(
        size=(num_modules, NUM_OF_TYPES_OF_MODULES),
        dtype=np.float32,
    )

    # "Connection" probability space
    conn_probability_space = rng.random(
        size=(num_modules, num_modules, NUM_OF_FACES),
        dtype=np.float32,
    )

    # "Rotation" probability space
    rotation_probability_space = rng.random(
        size=(num_modules, NUM_OF_ROTATIONS),
        dtype=np.float32,
    )

    # Decode the high-probability graph
    hpd = HighProbabilityDecoder(num_modules)
    return hpd.probability_matrices_to_graph(
        type_probability_space,
        conn_probability_space,
        rotation_probability_space,
    )

In [ ]:
POPULTION_SIZE = 100

In [ ]:
POPULATION_NODES = [ctk.node_from_graph(generate_random_individual()) for _ in range(POPULTION_SIZE)]

In [ ]:
config = ctk.SimilarityConfig()

In [ ]:
population_fingerprints = [ctk.collect_hash_fingerprint(node, config) for node in POPULATION_NODES]

In [ ]:
feature_series = ctk.series_from_population_fingerprint(population_fingerprints, space=ctk.VectorSpace.ENTIRE_ROBOT)
print(feature_series)

In [ ]:
cos_series = feature_series.cosine_similarity(inplace=False)
print(feature_series)
print(cos_series)

In [ ]:
print(cos_series[12])
cumul_series = cos_series.to_cumulative(inplace=False).normalize_by_radius()
print(cumul_series[12])

In [ ]:
cumul_series[3].sum_similarity_scores()